In [6]:
import torch
from torch import nn
from torch.nn import functional as F

In [17]:
class Residual(nn.Module):
  def __init__(self,in_channels,num_channels,conv_1x1=False,strides=1):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels,num_channels,kernel_size=3,padding=1,stride=strides)
    self.conv2 = nn.Conv2d(num_channels,num_channels,kernel_size=3,padding=1)
    if conv_1x1 == True:
      self.conv3 = nn.Conv2d(in_channels,num_channels,kernel_size=1,stride= strides)
    else:
      self.conv3 = None
    self.bn1 = nn.BatchNorm2d(num_channels)
    self.bn2 = nn.BatchNorm2d(num_channels)

  def forward(self,x):
    y = F.relu(self.bn1(self.conv1(x)))
    y = self.bn2(self.conv2(y))

    if self.conv3 != None:
      x = self.conv3(x)
    y += x

    return F.relu(y)


In [18]:
blk = Residual(3,3,conv_1x1=True,strides=2)
X = torch.rand(4, 3, 6, 6)
Y = blk(X)
Y.shape

torch.Size([4, 3, 3, 3])

In [33]:
b1=nn.Sequential(nn.Conv2d(1,64,kernel_size=7,stride=2,padding=3),
         nn.BatchNorm2d(64),
         nn.MaxPool2d(kernel_size=3,stride=2,padding=1))

def resnetblk(input_channels, num_channels, num_residuals,
                 first_block=False):
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(input_channels, num_channels,
                                conv_1x1=True, strides=2))
        else:
            blk.append(Residual(num_channels, num_channels))
    return blk

In [34]:
b2=nn.Sequential(*resnetblk(64,64,2,first_block=True))
b3=nn.Sequential(*resnetblk(64,128,2))
b4=nn.Sequential(*resnetblk(128,256,2))
b5=nn.Sequential(*resnetblk(256,512,2))

In [38]:
resnet = nn.Sequential(b1,b2,b3,b4,b5,
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten(),
            nn.Linear(512,10))

In [41]:
X = torch.rand(size=(1, 1, 224, 224))
for layer in resnet:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 64, 56, 56])
Sequential output shape:	 torch.Size([1, 64, 56, 56])
Sequential output shape:	 torch.Size([1, 128, 28, 28])
Sequential output shape:	 torch.Size([1, 256, 14, 14])
Sequential output shape:	 torch.Size([1, 512, 7, 7])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 512, 1, 1])
Flatten output shape:	 torch.Size([1, 512])
Linear output shape:	 torch.Size([1, 10])
